In [1]:
from utils import *
%load_ext autoreload
%autoreload 2

In [2]:
X_train, y_train, X_test = read_train_test(sentence_per_row_mode=True)

Reading training data ...
Reading training data : Done
Reading testing data ...
Reading testing data : Done


In [26]:
X_train["text_joined"] = [" ".join(X_train.text[i]) for i in range(X_train.shape[0])]

In [4]:
# y_train.loc[X_train[X_train.filename == "Bordeaux_605561.txt"].index]

In [3]:
sex_prediction = X_train.text.apply(naive_sex_classifier, args=(0.85, 1))

In [11]:
print("Nombre de fois où on se trompe de sexe : {}".format((sex_prediction != y_train.sexe).sum()))
print("Dont : {} femmes".format((y_train[(sex_prediction != y_train.sexe)].sexe == "femme").sum()))

Nombre de fois où on se trompe de sexe : 90
Dont : 61 femmes


***

In [42]:
print("Total de texte : {}".format(X_train.shape[0]))
print("Le nombre de fois où le mot 'victime' apparait dans le texte : {}".format(X_train.text_joined.str.contains("victime").sum()))
print("Le nombre de fois où le mot 'APPELANTE' apparait dans le texte : {}".format(X_train.text_joined.str.contains("APPELANTE").sum()))

Total de texte : 770
Le nombre de fois où le mot 'victime' apparait dans le texte : 728
Le nombre de fois où le mot 'APPELANTE' apparait dans le texte : 267


# Extraction du contexte autour de la date d'accident

In [59]:
def date_parser(date_string):
    """ Transforme une date de YYYY-mm-DD 
        en DD mm (en lettre) YYYY """
    
    year_month_day = date_string.split('-')
    
    list_Months=['Janvier',"Février",
                 'Mars','Avril','Mai','Juin','Juillet',
                 'Aout',"Août",'Septembre','Octobre','Novembre',
                 "Décembre"]
    list_months=['janvier',"février",
                 'mars','avril','mai','juin','juillet',
                 'aout',"août",'septembre','octobre','novembre',
                 'decembre',"décembre"]
    
    day = str(int(year_month_day[2])) # sans le 0 qui précède les unités
    Month = list_Months[int(year_month_day[1])-1]
    month = list_months[int(year_month_day[1])-1]
    year = year_month_day[0]
    
    return day + " " + Month + " " + year, day + " " + month + " " + year

In [130]:
def extract_X_sentences_before_after(text, date, X=2):
    
    try :
        capitalized_date, non_capitalized_date = date_parser(date)
    except :
        return "Date NC"
    
    L1 = [capitalized_date in sen for sen in text]
    L2 = [non_capitalized_date in sen for sen in text]
    
    if np.array(L1).any() :
        index = np.where(L1)[0][0]
    elif np.array(L2).any() :
        index = np.where(L2)[0][0]
    else :
        return "Pas trouvé de Date sous format DD mm (en lettre) YYYY dans le texte"
    
    result = text[index-X : index+X+1]
                  
    return result

def terms_to_graph(terms, window_size):
    '''This function returns a directed, weighted igraph from lists of list of terms (the tokens from the pre-processed text)
    e.g., ['quick','brown','fox']
    Edges are weighted based on term co-occurence within a sliding window of fixed size 'w'
    '''
    
    from_to = {}

    w = min(window_size, len(terms))
    # create initial complete graph (first w terms)
    terms_temp = terms[0:w]
    indexes = list(itertools.combinations(range(w), r=2))

    new_edges = []

    for my_tuple in indexes:
        new_edges.append(tuple([terms_temp[i] for i in my_tuple]))
    for new_edge in new_edges:
        if new_edge in from_to:
            from_to[new_edge] += 1
        else:
            from_to[new_edge] = 1

    # then iterate over the remaining terms
    for i in range(w, len(terms)):
        # term to consider
        considered_term = terms[i]
        # all terms within sliding window
        terms_temp = terms[(i - w + 1):(i + 1)]

        # edges to try
        candidate_edges = []
        for p in range(w - 1):
            candidate_edges.append((terms_temp[p], considered_term))

        for try_edge in candidate_edges:

            # if not self-edge
            if try_edge[1] != try_edge[0]:

                # if edge has already been seen, update its weight
                if try_edge in from_to:
                    from_to[try_edge] += 1

                # if edge has never been seen, create it and assign it a unit weight
                else:
                    from_to[try_edge] = 1

    # create empty graph
    g = igraph.Graph(directed=True)

    # add vertices
    g.add_vertices(sorted(set(terms)))

    # add edges, direction is preserved since the graph is directed
    g.add_edges(list(from_to.keys()))

    # set edge and vertice weights
    g.es['weight'] = list(from_to.values()) # based on co-occurence within sliding window
    g.vs['weight'] = g.strength(weights=list(from_to.values())) # weighted degree

    return (g)

def core_dec(g,weighted):
    '''(un)weighted k-core decomposition'''
    # work on clone of g to preserve g 
    gg = copy.deepcopy(g)
    if not weighted:
        gg.vs['weight'] = gg.strength() # overwrite the 'weight' vertex attribute with the unweighted degrees
    # initialize dictionary that will contain the core numbers
    cores_g = dict(zip(gg.vs['name'],[0]*len(gg.vs)))
    
    while len(gg.vs) > 0:
        # find index of lowest degree vertex
        min_degree = min(gg.vs['weight'])
        index_top = gg.vs['weight'].index(min_degree)
        name_top = gg.vs[index_top]['name']
        # get names of its neighbors
        neighbors = gg.vs[gg.neighbors(index_top)]['name']
        # exclude self-edges
        neighbors = [elt for elt in neighbors if elt!=name_top]
        # set core number of lowest degree vertex as its degree
        cores_g[name_top] = min_degree
        # delete top vertex and its incident edges
        gg.delete_vertices(index_top)
        
        if neighbors:
            if weighted: 
                new_degrees = gg.strength(weights=gg.es['weight'])
            else:
                new_degrees = gg.strength()
            # iterate over neighbors of top element
            for neigh in neighbors:
                index_n = gg.vs['name'].index(neigh)
                gg.vs[index_n]['weight'] = max(min_degree,new_degrees[index_n])  
        
    return(cores_g)


In [131]:
context_to_analyse = extract_X_sentences_before_after(X_train.text[0], y_train.date_accident[0], X=4)
context_to_analyse = [sen for sen in context_to_analyse if sen]

In [136]:
context_to_analyse

['d’autre part,',
 'DIRECTION REGIONALE DES AFFAIRES SANITAIRES ET SOCIALES AQUITAINE Cité Administrative BP 952 Rue Jules Ferry 33063 BORDEAUX CEDEX NI PRESENTE, NI REPRESENTEE PARTIE INTERVENANTE : A rendu l’arrêt réputé contradictoire suivant. La cause a été débattue et plaidée en audience publique le 19 Mars 2002 devant Monsieur MILHET, Président de Chambre, Madame LATRABE, Conseiller, Monsieur ROS, Conseiller, assistés de Nicole GALLOIS, Greffier et après qu’il en ait été délibéré par les magistrats du siège ayant assisté aux débats, les parties ayant été avisées de la date à laquelle l’arrêt serait rendu. * * * ',
 'Le 9 avril 1991, Monsieur X... a été victime d’un accident du travail au cours duquel il a été gravement blessé. ',
 'Suivant jugement en date du 24 avril 1997 confirmé par arrêt de la Cour en date du 30 mars 1999, le Tribunal des Affaires de Sécurité Sociale de LOT et GARONNE a dit que cet accident était dû à la faute',
 'inexcusable de la S.A.R.L. TRANSPORTS MOREL, 

In [132]:
words = ' '.join(context_to_analyse)

In [142]:
words

'd’autre part, DIRECTION REGIONALE DES AFFAIRES SANITAIRES ET SOCIALES AQUITAINE Cité Administrative BP 952 Rue Jules Ferry 33063 BORDEAUX CEDEX NI PRESENTE, NI REPRESENTEE PARTIE INTERVENANTE : A rendu l’arrêt réputé contradictoire suivant. La cause a été débattue et plaidée en audience publique le 19 Mars 2002 devant Monsieur MILHET, Président de Chambre, Madame LATRABE, Conseiller, Monsieur ROS, Conseiller, assistés de Nicole GALLOIS, Greffier et après qu’il en ait été délibéré par les magistrats du siège ayant assisté aux débats, les parties ayant été avisées de la date à laquelle l’arrêt serait rendu. * * *  Le 9 avril 1991, Monsieur X... a été victime d’un accident du travail au cours duquel il a été gravement blessé.  Suivant jugement en date du 24 avril 1997 confirmé par arrêt de la Cour en date du 30 mars 1999, le Tribunal des Affaires de Sécurité Sociale de LOT et GARONNE a dit que cet accident était dû à la faute inexcusable de la S.A.R.L. TRANSPORTS MOREL, employeur de Mons

In [115]:
import itertools
import igraph
import copy

In [146]:
g = terms_to_graph(words.split(" "), 3)
dico = core_dec(g, True)

In [144]:
words

'd’autre part, DIRECTION REGIONALE DES AFFAIRES SANITAIRES ET SOCIALES AQUITAINE Cité Administrative BP 952 Rue Jules Ferry 33063 BORDEAUX CEDEX NI PRESENTE, NI REPRESENTEE PARTIE INTERVENANTE : A rendu l’arrêt réputé contradictoire suivant. La cause a été débattue et plaidée en audience publique le 19 Mars 2002 devant Monsieur MILHET, Président de Chambre, Madame LATRABE, Conseiller, Monsieur ROS, Conseiller, assistés de Nicole GALLOIS, Greffier et après qu’il en ait été délibéré par les magistrats du siège ayant assisté aux débats, les parties ayant été avisées de la date à laquelle l’arrêt serait rendu. * * *  Le 9 avril 1991, Monsieur X... a été victime d’un accident du travail au cours duquel il a été gravement blessé.  Suivant jugement en date du 24 avril 1997 confirmé par arrêt de la Cour en date du 30 mars 1999, le Tribunal des Affaires de Sécurité Sociale de LOT et GARONNE a dit que cet accident était dû à la faute inexcusable de la S.A.R.L. TRANSPORTS MOREL, employeur de Mons

In [147]:
dico

{'': 4.0,
 '*': 4.0,
 '19': 4.0,
 '1991,': 4.0,
 '1997': 4.0,
 '1999,': 4.0,
 '2002': 4.0,
 '24': 4.0,
 '30': 4.0,
 '33063': 2.0,
 '9': 4.0,
 '952': 2.0,
 ':': 3.0,
 'A': 3.0,
 'A....': 3.0,
 'AFFAIRES': 2.0,
 'AQUITAINE': 2.0,
 'Administrative': 2.0,
 'Affaires': 4.0,
 'BORDEAUX': 2.0,
 'BP': 2.0,
 'CEDEX': 2.0,
 'Chambre,': 4.0,
 'Cité': 2.0,
 'Conseiller,': 4.0,
 'Cour': 4.0,
 'DES': 2.0,
 'DIRECTION': 2.0,
 'ET': 2.0,
 'Ferry': 2.0,
 'GALLOIS,': 4.0,
 'GARONNE': 4.0,
 'Greffier': 4.0,
 'INTERVENANTE': 3.0,
 'Jules': 2.0,
 'LATRABE,': 4.0,
 'LOT': 4.0,
 'La': 3.0,
 'Le': 4.0,
 'MILHET,': 4.0,
 'MOREL,': 4.0,
 'Madame': 4.0,
 'Mars': 4.0,
 'Monsieur': 4.0,
 'NI': 3.0,
 'Nicole': 4.0,
 'PARTIE': 3.0,
 'PRESENTE,': 3.0,
 'Président': 4.0,
 'REGIONALE': 2.0,
 'REPRESENTEE': 3.0,
 'ROS,': 4.0,
 'Rue': 2.0,
 'S.A.R.L.': 4.0,
 'SANITAIRES': 2.0,
 'SOCIALES': 2.0,
 'Sociale': 4.0,
 'Suivant': 4.0,
 'Sécurité': 4.0,
 'TRANSPORTS': 4.0,
 'Tribunal': 4.0,
 'X...': 4.0,
 'a': 4.0,
 'accident': 